# Imports

In [30]:
! pip install -r requirements.txt

In [63]:
import os
import cv2
import random
import numpy as np
import uuid
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Layer, Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.metrics import Precision, Recall

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print("Memory growth must be set before GPUs have been initialized")
        print(e)
else:
    print("No GPU available")

# Directory Configs

In [32]:
project_path = os.path.curdir

data_path = os.path.join(project_path, 'data')
os.makedirs(data_path, exist_ok=True)

# Directorios
matched_path = os.path.join(data_path, 'matched')
unmatched_path = os.path.join(data_path,'unmatched')
samples_path = os.path.join(data_path, 'samples')
dataset_path = os.path.join(project_path, 'dataset')
app_data_path = os.path.join(project_path,'application_data')
verific_path = os.path.join(app_data_path,'verification_images')
input_image_path = os.path.join(app_data_path, 'input_image')
save_model_path = os.path.join(project_path, 'save_model')
checkpoint_path = os.path.join(project_path,'checkpoints')

os.makedirs(dataset_path, exist_ok=True)
os.makedirs(matched_path, exist_ok=True)
os.makedirs(unmatched_path, exist_ok=True)
os.makedirs(samples_path, exist_ok=True)
os.makedirs(app_data_path,exist_ok=True)
os.makedirs(verific_path,exist_ok=True)
os.makedirs(input_image_path,exist_ok=True)
os.makedirs(checkpoint_path, exist_ok=True)
os.makedirs(save_model_path, exist_ok=True)

!tree -L 2


.
├── application_data
│   ├── input_image
│   └── verification_images
├── checkpoints
├── data
│   ├── matched
│   ├── samples
│   └── unmatched
├── dataset
│   └── lfw.tgz
├── face_recognition.ipynb
├── README.md
├── requirements.txt
└── save_model

11 directories, 4 files


# Dataset download

In [33]:
if not os.path.exists(os.path.join(dataset_path, 'lfw.tgz')):
    !wget http://vis-www.cs.umass.edu/lfw/lfw.tgz -P {dataset_path}
else:
    print('The file lfw.tgz is already downloaded.')


The file lfw.tgz is already downloaded.


# Checking and Extracting Unmatched Directory Contents

In [34]:
if len(os.listdir(unmatched_path)) > 0:
    print(f'The "unmatched" directory already contains {len(os.listdir(unmatched_path))} files:')
    print('\n'.join(os.listdir(unmatched_path)[:10]))
else:
    !tar -xzf {os.path.join(dataset_path, 'lfw.tgz')} -C {unmatched_path} --strip-components 1
    print('File extracted in the "unmatched" directory.')


The "unmatched" directory already contains 13233 files:
Sanjay_Gupta_0001.jpg
Erik_Morales_0002.jpg
Nadia_Petrova_0002.jpg
Gerhard_Schroeder_0028.jpg
George_W_Bush_0101.jpg
Zhu_Rongji_0009.jpg
Gary_Condit_0001.jpg
Yusuf_Misbac_0001.jpg
Junichiro_Koizumi_0031.jpg
Alvaro_Uribe_0022.jpg


# Moving Files from Subdirectories to the Unmatched Directory

In [35]:
if any(os.path.isdir(os.path.join(unmatched_path, d)) for d in os.listdir(unmatched_path)):
    print("Moving files from subdirectories to the 'unmatched' directory...")
    for subdir in os.listdir(unmatched_path):
        subdir_path = os.path.join(unmatched_path, subdir)
        if os.path.isdir(subdir_path):
            for filename in os.listdir(subdir_path):
                src = os.path.join(subdir_path, filename)
                dst = os.path.join(unmatched_path, filename)
                os.replace(src, dst)
            os.rmdir(subdir_path)
    print("Files moved and folders deleted.")
else:
    num_images = len(os.listdir(unmatched_path))
    print(f"There are {num_images} images in the 'unmatched' directory and no subdirectories found.")

There are 13233 images in the 'unmatched' directory and no subdirectories found.


# Collect Images 


In [36]:
def collect_images(samples_path, matched_path):
    cap = cv2.VideoCapture(0)
    count_samples = 0
    count_matched = 0

    while cap.isOpened():
        ret, frame = cap.read()
        frame = frame[120:120+250, 200:200+250, :]
        cv2.imshow('Image Collection', frame)
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('s'):
            imgname = os.path.join(samples_path, f'{uuid.uuid1()}.jpg')
            cv2.imwrite(imgname, frame)
            count_samples += 1
            print(f'Samples image saved: {imgname}')
        elif key == ord('a'):
            imgname = os.path.join(matched_path, f'{uuid.uuid1()}.jpg')
            cv2.imwrite(imgname, frame)
            count_matched += 1
            print(f'Matched image saved: {imgname}')
        elif key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return count_samples, count_matched

In [37]:
collect_images(samples_path,matched_path)

QObject::moveToThread: Current thread (0x7e1a2b0) is not the object's thread (0x8796620).
Cannot move to target thread (0x7e1a2b0)

QObject::moveToThread: Current thread (0x7e1a2b0) is not the object's thread (0x8796620).
Cannot move to target thread (0x7e1a2b0)

QObject::moveToThread: Current thread (0x7e1a2b0) is not the object's thread (0x8796620).
Cannot move to target thread (0x7e1a2b0)

QObject::moveToThread: Current thread (0x7e1a2b0) is not the object's thread (0x8796620).
Cannot move to target thread (0x7e1a2b0)

QObject::moveToThread: Current thread (0x7e1a2b0) is not the object's thread (0x8796620).
Cannot move to target thread (0x7e1a2b0)

QObject::moveToThread: Current thread (0x7e1a2b0) is not the object's thread (0x8796620).
Cannot move to target thread (0x7e1a2b0)

QObject::moveToThread: Current thread (0x7e1a2b0) is not the object's thread (0x8796620).
Cannot move to target thread (0x7e1a2b0)

QObject::moveToThread: Current thread (0x7e1a2b0) is not the object's thread

(0, 0)

# Preparing Datasets for Training

In [38]:
def validate_image_count(desired_count, *directories):
    for directory in directories:
        current_count = len(os.listdir(directory))
        if current_count < desired_count:
            raise ValueError(f'A minimum of {desired_count} images are required in {directory}, but only {current_count} were found.')
    

In [39]:

# TODO: Modify the 'num_images' variable to adjust the number of images to be processed.
num_images = 300 

validate_image_count (num_images,matched_path,samples_path,unmatched_path) # type: ignore

In [40]:
# Preparación de datos para entrenamiento
matched_dataset = tf.data.Dataset.list_files(matched_path + '/*.jpg').take(num_images)
samples_dataset = tf.data.Dataset.list_files(samples_path + '/*.jpg').take(num_images)
unmatched_dataset = tf.data.Dataset.list_files(unmatched_path + '/*.jpg').take(num_images)


# Preprocess Image Function

In [41]:
def preprocess_image(image_path):
  
    byte_img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(byte_img, channels=3)  # Asegura que la imagen tenga 3 canales (RGB)
    img = tf.image.resize(img, (105, 105))
    img = img / 255.0  # Normaliza los valores de los píxeles a [0, 1]

    return img

# Data Pipeline

## Creation of the Training Dataset

In [42]:
## Creation of the Training Dataset

positives = tf.data.Dataset.zip((
    samples_dataset,
    matched_dataset,
    tf.data.Dataset.from_tensor_slices(tf.ones(len(samples_dataset)))  # Label '1' for positive pairs
))

negatives = tf.data.Dataset.zip((
    samples_dataset,
    unmatched_dataset,
    tf.data.Dataset.from_tensor_slices(tf.zeros(len(samples_dataset)))  # Label '0' for negative pairs
))

# Combine positive and negative datasets
data = positives.concatenate(negatives)

## Define the pair preprocessing function

In [43]:
def preprocess_pair(sample_image, validation_image, label):
    return preprocess_image(sample_image), preprocess_image(validation_image), label

## Apply the preprocessing function to the dataset:

In [44]:
data = data.map(preprocess_pair) # Aplica la función de preprocesamiento al dataset 
data = data.cache() #Almacena en caché el dataset para mejorar la eficiencia.
data = data.shuffle(buffer_size=1024) #Baraja el dataset con un buffer de 1024 elementos.

## Split the dataset into training and testing sets:

### Train dataset

In [45]:
# Toma el 70% del conjunto de datos para entrenamiento
train_data = data.take(round(len(data) * 0.7))

# Optimización de Recursos

# Agrupa los elementos en lotes de 16
train_data = train_data.batch(16)

# Superpone el procesamiento y entrenamiento con prefetch de 8 lotes
train_data = train_data.prefetch(8)


### Test dataset

In [46]:

test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

# Embedding model definition

In [47]:

def make_embedding(): 
    inp = Input(shape=(105, 105, 3), name='input_image')
    
    c1 = Conv2D(64, (10, 10), activation='relu')(inp)
    m1 = MaxPooling2D(pool_size=(2, 2), padding='same')(c1)
    
    c2 = Conv2D(128, (7, 7), activation='relu')(m1)
    m2 = MaxPooling2D(pool_size=(2, 2), padding='same')(c2)
    
    c3 = Conv2D(128, (4, 4), activation='relu')(m2)
    m3 = MaxPooling2D(pool_size=(2, 2), padding='same')(c3)
    
    c4 = Conv2D(256, (4, 4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    
    return Model(inputs=[inp], outputs=d1, name='embedding')

# Se define una red neuronal convolucional (CNN) para extraer embeddings (representaciones de características) de imágenes.


## Distancia L1


In [48]:
# Se define una capa personalizada para calcular la distancia L1 (la suma de las diferencias absolutas) entre dos vectores.

class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
       
    def call(self, inputs):
        return tf.abs(inputs[0] - inputs[1])
    

# Siamese Model

In [49]:
def make_siamese_model(): 
    input_sample_image = Input(shape=(105, 105, 3), name='sample_image')
    input_validation_image = Input(shape=(105, 105, 3), name='validation_image')
    
    embedding_model = make_embedding()
    
    encoded_s = embedding_model(input_sample_image)
    encoded_v = embedding_model(input_validation_image)
    
    distance = L1Dist(name='distance')([encoded_s, encoded_v])
    
    classifier = Dense(1, activation='sigmoid')(distance)
    
    model = Model(inputs=[input_sample_image, input_validation_image], outputs=classifier, name='siamese_network')
    
    return model

In [50]:
siamese_model = make_siamese_model()
siamese_model.summary()



2024-08-01 12:05:16.525605: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 150994944 exceeds 10% of free system memory.
2024-08-01 12:05:17.528085: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 150994944 exceeds 10% of free system memory.


Model: "siamese_network"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ sample_image        │ (None, 105, 105,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ validation_image    │ (None, 105, 105,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 4096)      │ 38,960,448 │ sample_image[0][… │
│ (Functional)        │                   │            │ validation_image… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l1_dist_1 (L1Dist)  │ (None, 4096)      │          0 │ embedding[0][0],  │
│                     │                   │            │ embedding[1][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │      4,097 │ l1_dist_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 38,964,545 (148.64 MB)

 Trainable params: 38,964,545 (148.64 MB)

 Non-trainable params: 0 (0.00 B)

### ***Note :***
> ***I could have chosen a simpler approach using Keras' compile function, but I decided to implement the training process manually.
This allowed me to understand in detail each step involved in training the model. Although I haven't tested the following code,
it should work fine. Here's the streamlined approach:***

```python

siamese_model = make_siamese_model()
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision(), Recall()])

history = siamese_model.fit(train_data, 
                            validation_data=test_data,
                            epochs=5,
                            batch_size=16)

plt.plot(history.history['precision'], label='Precision')
plt.plot(history.history['recall'], label='Recall')
plt.title('Precision and Recall of Siamese Model')
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.legend()
plt.show()
```

In [51]:
# Define binary cross-entropy loss function
binary_cross_entropy_loss = tf.losses.BinaryCrossentropy()

# Initialize Adam optimizer with a learning rate of 0.0001
adam_optimizer = tf.keras.optimizers.Adam(1e-4)

checkpoint_prefix = os.path.join(checkpoint_path, 'ckpt')
model_checkpoint = tf.train.Checkpoint(opt=adam_optimizer, siamese_model=siamese_model)


In [52]:

@tf.function
def training_step(batch):
    # Record all operations for automatic differentiation
    with tf.GradientTape() as tape:
        # Extract images and labels from the batch
        input_images = batch[:2]  # Anchor and positive/negative images
        true_labels = batch[2]    # Labels
        
        # Perform a forward pass through the Siamese model
        predicted_labels = siamese_model(input_images, training=True)
        
        # Compute the binary cross-entropy loss
        loss_value = binary_cross_entropy_loss(true_labels, predicted_labels)
        print(loss_value)  # Print the loss for monitoring
        
    # Compute gradients of the loss with respect to model variables
    gradients = tape.gradient(loss_value, siamese_model.trainable_variables)
    
    # Apply gradients to update model weights
    adam_optimizer.apply_gradients(zip(gradients, siamese_model.trainable_variables))
    
    # Return the computed loss for further analysis
    return loss_value

In [53]:
def train_model(training_data, num_epochs):
    # Loop through each epoch
    for current_epoch in range(1, num_epochs + 1):
        print('\n Epoch {}/{}'.format(current_epoch, num_epochs))
        progress_bar = tf.keras.utils.Progbar(len(training_data))
        
        # Create metric objects for evaluation
        recall_metric = Recall()
        precision_metric = Precision()
        
        # Loop through each batch in the training data
        for batch_index, current_batch in enumerate(training_data):
            # Execute the training step for the current batch
            loss_value = training_step(current_batch)
            
            # Make predictions using the Siamese model
            predicted_labels = siamese_model.predict(current_batch[:2])
            
            # Update recall and precision metrics
            recall_metric.update_state(current_batch[2], predicted_labels)
            precision_metric.update_state(current_batch[2], predicted_labels) 
            
            # Update the progress bar
            progress_bar.update(batch_index + 1)
        
        # Print the loss, recall, and precision for the current epoch
        print(loss_value.numpy(), recall_metric.result().numpy(), precision_metric.result().numpy())
        
        # Save checkpoints every 10 epochs
        if current_epoch % 10 == 0: 
            model_checkpoint.save(file_prefix=checkpoint_prefix)


In [54]:
train = True 
if(train):
    train_model(train_data,50)

In [55]:
# Inicializar métricas de Recuperación y Precisión para la evaluación sobre los datos de prueba
recall_metric = Recall()
precision_metric = Precision()

for sample_image, validate_image, true_label in test_data.as_numpy_iterator():
    # Obtener las predicciones del modelo siamesa
    predict_label = siamese_model.predict([sample_image,validate_image])
    # Actualizar la métrica de Recuperación con las etiquetas verdaderas y las predicciones
    recall_metric.update_state(true_label, predict_label)
    # Actualizar la métrica de Precisión con las etiquetas verdaderas y las predicciones
    precision_metric.update_state(true_label, predict_label)


# Print final recall and precision results
print(f'Recall: {recall_metric.result().numpy()}, Precision: {precision_metric.result().numpy()}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Recall: 0.13684210181236267, Precision: 0.21311475336551666


2024-08-01 12:06:29.293565: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [56]:
def load_siamese_model(save_model_path, date_str=None):
    custom_objects = {
        'L1Dist': L1Dist,
        'BinaryCrossentropy': tf.losses.BinaryCrossentropy
    }

    if date_str:
        # Si se proporciona una fecha, intenta cargar el modelo correspondiente
        model_filename = f"siamese_model_{date_str}.h5"
        model_filepath = os.path.join(save_model_path, model_filename)
        if os.path.exists(model_filepath):
            print(f"Cargando el modelo desde: {model_filepath}")
            return tf.keras.models.load_model(model_filepath, custom_objects=custom_objects)
        else:
            raise FileNotFoundError(f"No se encontró el modelo para la fecha proporcionada: {date_str}")
    else:
        # Si no se proporciona una fecha, carga el modelo más reciente
        model_files = [f for f in os.listdir(save_model_path) if f.startswith("siamese_model_") and f.endswith(".h5")]
        if not model_files:
            raise FileNotFoundError("No se encontraron modelos en el directorio especificado.")
        
        # Ordenar los archivos por fecha y hora en el nombre del archivo
        model_files.sort(key=lambda x: datetime.datetime.strptime(x.split("_", 2)[2].split(".")[0], "%Y_%m_%d_%H_%M"), reverse=True)
        latest_model_filename = model_files[0]
        latest_model_filepath = os.path.join(save_model_path, latest_model_filename)
        print(f"Cargando el modelo más reciente desde: {latest_model_filepath}")
        return tf.keras.models.load_model(latest_model_filepath, custom_objects=custom_objects)


In [57]:

current_datetime = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")
model_filename = f"siamese_model_{current_datetime}.h5"

# Guardar el modelo en el subdirectorio
siamese_model.save(os.path.join(save_model_path,model_filename))

In [58]:
# Para cargar un modelo específico por fecha
# model = load_siamese_model(save_model_path, "2024_08_01_12_00")

# Para cargar el modelo más reciente
try:
    siamese_model = load_siamese_model(save_model_path)
except Exception as e:
    print(f'Error loading model: {e}')

Cargando el modelo más reciente desde: ./save_model/siamese_model_2024_08_01_12_06.h5


In [59]:
# View model summary
siamese_model.summary()

Model: "siamese_network"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ sample_image        │ (None, 105, 105,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ validation_image    │ (None, 105, 105,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 4096)      │ 38,960,448 │ sample_image[0][… │
│ (Functional)        │                   │            │ validation_image… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l1_dist_2 (L1Dist)  │ (None, 4096)      │          0 │ embedding[0][0],  │
│                     │                   │            │ embedding[1][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │      4,097 │ l1_dist_2[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 38,964,545 (148.64 MB)

 Trainable params: 38,964,545 (148.64 MB)

 Non-trainable params: 0 (0.00 B)

In [60]:
def verify(model, detection_threshold, verification_threshold):
    # Build results array
    results = []
    input_img = preprocess_image(os.path.join(input_image_path,'input_image.jpg'))
    
    for image in verific_path:
        validation_img = preprocess_image(os.path.join(verific_path,image))
        
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
    
    # Detection Threshold: Metric above which a prediciton is considered positive 
    detection = np.sum(np.array(results) > detection_threshold)
    
    # Verification Threshold: Proportion of positive predictions / total positive samples 
    verification = detection / len(os.listdir(os.path.join(verific_path))) 
    verified = verification > verification_threshold
    
    return results, verified

In [61]:
cap = cv2.VideoCapture(1)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    
    cv2.imshow('Verification', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('v'):
  
        cv2.imwrite(os.path.join(input_image_path,'input_image.jpg'), frame)
        # Run verification
        results, verified = verify(siamese_model, 0.5, 0.5)
        print(verified)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

[ WARN:0@1084.194] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@1084.420] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


In [62]:

# Mostrar los resultados
print('Número de resultados mayores a 0.9:', np.sum(np.squeeze(results) > 0.9))
print('Resultados:', results)


NameError: name 'results' is not defined